# Lab 6 - Arend Wong (aw3146) - 2021-05-29

### 1. Run a naive ("pooled") OLS regression on the panel data. Tell we how you expect your Xs to affect your Y and why. Interpret your results.
### 2. Run a first differences regression on the same model in Question 1. Interpret your results. Do you draw a different conclusion than in Question 1? Explain.¶

In [2]:
# importing relevant modules...
from __future__ import division
import pandas as pd
import numpy as np
import re, os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns #os.system('pip install seaborn')
from scipy import stats

try:
    import linearmodels as lm
except:
    os.system("pip install linearmodels")
    import linearmodels as lm

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

# defining helper functions for later in my code
def recode_onehot_if(s='CHILDRENS COAT', pattern='CHILDREN'):
    result = '1' if pattern in str(s) else '0'
    return(result)

### I'm using the GSS panel data, but different variables than Eirich used!

In [204]:
g = pd.read_csv("courseworks/panel-for-R.csv")
print('Columns in dataset (g):\n', g.columns.tolist())
g

Columns in dataset (g):
 ['idnum', 'panelwave', 'ballot', 'form', 'formwt', 'oversamp', 'sample', 'panstat_2', 'panstat_3', 'mar1', 'mar2', 'mar3', 'mar4', 'mar5', 'mar6', 'mar7', 'mar8', 'mar11', 'mar12', 'abany', 'abdefect', 'abhlth', 'abnomore', 'abpoor', 'abrape', 'absingle', 'acqntsex', 'adults', 'advfront', 'affrmact', 'age', 'aged', 'agekdbrn', 'astrolgy', 'astrosci', 'attend', 'babies', 'balneg', 'balpos', 'bible', 'bigbang', 'bizbstgw', 'bizbsttx', 'bizinfgw', 'bizinftx', 'blkwhite', 'born', 'boyorgrl', 'buyvalue', 'cappun', 'caremost', 'childs', 'chldidel', 'class', 'closeblk', 'closewht', 'cohort', 'colath', 'colcom', 'coldeg1', 'colhomo', 'colmil', 'colrac', 'colsci', 'colscinm', 'comorsci', 'compperf', 'comprend', 'compwage', 'conarmy', 'conbus', 'conclerg', 'condom', 'condrift', 'coneduc', 'confed', 'confinan', 'coninc', 'conjudge', 'conlabor', 'conlegis', 'conmedic', 'conpress', 'conrinc', 'consci', 'consent', 'conterrr', 'contv', 'coop', 'courts', 'crack30', 'dateintv',

,idnum,panelwave,ballot,form,formwt,oversamp,sample,panstat_2,panstat_3,mar1,...,wtpan12,wtpan123,wtpannr12,wtpannr123,xmarsex,xmovie,xnorcsiz,year,yearval,zodiac
0,9,1,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,2.0,1.0,2006.0,NaN,9.0
1,9,2,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2008.0,NaN,9.0
2,9,3,3,2,1,1,9,1,1,1.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2010.0,NaN,9.0
3,10,1,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2006.0,NaN,3.0
4,10,2,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2008.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,4509,2,2,1,1,1,9,1,1,1.0,...,1.888013,1.692214,1.982779,1.924745,NaN,2.0,4.0,2008.0,NaN,3.0
5996,4509,3,2,1,1,1,9,1,1,1.0,...,1.888013,1.692214,1.982779,1.924745,NaN,2.0,4.0,2010.0,NaN,3.0
5997,4510,1,2,2,1,1,9,1,1,3.0,...,0.944007,1.095132,1.093349,1.080810,NaN,2.0,4.0,2006.0,NaN,11.0
5998,4510,2,2,2,1,1,9,1,1,3.0,...,0.944007,1.095132,1.093349,1.080810,NaN,2.0,4.0,2008.0,NaN,11.0


In [210]:
# print(g.columns[g.columns.str.contains('condom|partner|belief|feel|emp|wrk|sex|edu|frend|gender')])
g.panelwave.value_counts()
g.sexfreq.value_counts()
g['sexweeklyormore'] = g.sexfreq.astype(str).replace({'1.0':'0','2.0':'0','3.0':'0','4.0':'1', '5.0':'1', '6.0':'1', '0.0':'0'}, regex=False).apply(lambda s: float(s) if s!='nan' else float(np.nan)).astype(float)
print('\nsexweeklyormore levels:\n', g.sexweeklyormore.value_counts())


sexweeklyormore levels:
 0.0    1773
1.0    1134
Name: sexweeklyormore, dtype: int64


## 1. Run a naive ("pooled") OLS regression on the panel data.  Tell we how you expect your Xs to affect your Y and why.   Interpret your results.

## The question is net of the respondents' gender and number of sexual partners in the past year and relationship status (single or not), does _*increased frequency of sex*_ predict people's general happiness?

### "General happiness" is operationalized on a scale from 1-3, where 1 means VERY HAPPY and 3 means NOT TOO HAPPY.

Other variables used: partners (meaning number of sexual partners on a 1-4 scale); sex (meaning gender of female or male); absingle (meaning whether the respondent's relationship status is single or not).

In [202]:
lm_ols = smf.ols(formula = 'happy ~ sexweeklyormore + partners + sex + absingle', subset=(g['sexweeklyormore']!='nan'), data = g).fit()
print (lm_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  happy   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     5.169
Date:                Mon, 14 Jun 2021   Prob (F-statistic):           0.000384
Time:                        22:56:35   Log-Likelihood:                -1855.3
No. Observations:                1941   AIC:                             3721.
Df Residuals:                    1936   BIC:                             3749.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.9818      0.071     

### Whoa! It appears that increased frequency of sex does predict general happiness, however, there is an inverse relationship between the two. That's actually what I expected. I thought that having sex more often would likely predict increased happiness because of a multitude of factors, including increased endorphins and sense of closeness to an individual, etc...

### Respondents who have sex weekly or more often are (on average) .1148 units MORE generally happy (less generally unhappy) on the 1-3 reverse happiness scale--statistically significantly, regardless of their gender, number of sexual partners, and relationship status (b=-0.1148; p<.001).

## 2. Run a first differences regression on the same model in Question 1.  Interpret your results.  Do you draw a different conclusion than in Question 1?  Explain.

In [203]:
g0 = g.query("sexfreq < 7").dropna(subset=['happy', 'sexweeklyormore', 'partners', 'sex', 'absingle']).query("sexweeklyormore != 'nan'").set_index(['idnum', 'panelwave'])
lm_fd = FirstDifferenceOLS.from_formula('happy ~ sexweeklyormore + partners + sex + absingle', g0)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  happy   R-squared:                        0.0066
Estimator:         FirstDifferenceOLS   R-squared (Between):             -0.1747
No. Observations:                 733   R-squared (Within):               0.0066
Date:                Mon, Jun 14 2021   R-squared (Overall):             -0.1698
Time:                        22:56:38   Log-likelihood                   -747.37
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.2165
Entities:                        1208   P-value                           0.3023
Avg Obs:                       1.6068   Distribution:                   F(4,729)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             1.4189
                            

## When we look to see what happens if someone switches from having sex more than once per week to having sex less than weekly, they do become slightly unhappier (-0.0986 is less happy/more unhappy than the previous effect we measured), but only marginally significantly so (p=0.0517). The switching of the values of the other Xs has no statistically significant impact on general happiness.
